In [ ]:
"""danielsinkin97@gmail.com"""

import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, FloatSlider

from computer_vision.src.filter import FilterType, apply_filter, get_filter
from computer_vision.src.autocorrelation import get_autocorrelation_error_matrix, get_autocorrelation_matrix
from computer_vision.util.images import load_image_as_array, rgb_to_grayscale

image = rgb_to_grayscale(
    load_image_as_array(
        "/Users/danielsinkin/GitHub_private/computer-vision/data/hummingbird.png"
    )
)

i_h, i_w = image.shape

filter_ = np.array([-2, -1, 0, 1, 2], np.float32)
ac_matrix = get_autocorrelation_matrix(
    image=image,
    filter_hori=filter_,
    filter_vert=filter_.reshape(-1, 1),
    filter_w=get_filter(FilterType.GAUSS_5X5),
)

delta_u = np.array([10, -5], dtype=np.float32)
ac_error_matrix = get_autocorrelation_error_matrix(ac_matrix, delta_u)

interest_highlighting = apply_filter(
    ac_error_matrix, get_filter(FilterType.GAUSS_15X15), pad_same_size=True
)
interest_highlighting = (
    (interest_highlighting - interest_highlighting.min())
    / (interest_highlighting.max() - interest_highlighting.min())
    * 255
)

@interact(d=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.4, description="Blend d"))
def update(d=0.4):
    blended = d * image + (1 - d) * interest_highlighting

    fig, axs = plt.subplots(1, 3, figsize=(12, 8))

    axs[0].imshow(blended, cmap="viridis", interpolation="nearest")
    axs[0].set_title("Viridis", color="white")
    axs[0].axis("off")

    axs[1].imshow(blended, cmap="magma", interpolation="nearest")
    axs[1].set_title("Magma", color="white")
    axs[1].axis("off")

    axs[2].imshow(blended, cmap="gray", interpolation="nearest")
    axs[2].set_title("Gray", color="white")
    axs[2].axis("off")

    fig.tight_layout()
    fig.patch.set_facecolor("black")
    plt.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

rows, cols = ac_error_matrix.shape
x = np.arange(cols)
y = np.arange(rows)
X, Y = np.meshgrid(x, y)

fig = go.Figure(data=[go.Surface(z=ac_error_matrix, x=X, y=Y, colorscale="Viridis")])

fig.update_layout(
    title="AC Error Matrix Surface",
    scene=dict(
        xaxis_title="Column Index",
        yaxis_title="Row Index",
        zaxis_title="Error Value",
    ),
    autosize=True,
    margin=dict(l=0, r=0, b=0, t=50),
)

fig.show(renderer="browser")